# CCI API Demo

## Setup

In [ ]:
import requests

# Import JSON for pretty printing output
import json

OPENSEARCH_URL = 'https://archive.opensearch.ceda.ac.uk/opensearch'


def pprint(input: dict)-> None:
    """pretty prints a dictionary"""
    
    print(json.dumps(input, indent=4))
    
def get_opensearch_request(url: str) -> requests.models.Response:
    
    return requests.get(url, headers={'Accept':'application/geo+json'})

## Take a look at the top level collections

In [ ]:
# Retrieve top level catalog
catalogs = get_opensearch_request(f'{OPENSEARCH_URL}/request')
print(catalogs.status_code)
pprint(catalogs.json())


## Retrieve the description document for CCI

In [ ]:
# Get first collection
first_collection = catalogs.json()['features'][0]

# Extract the description URL
description_url = first_collection['properties']['links']['search'][0]['href'] 
cci_description = get_opensearch_request(description_url)

import xml.etree.ElementTree as ET
osdd = ET.fromstring(cci_description.text)

print(osdd.tag,osdd.text)
for child in osdd:
    print(child.tag, child.attrib, child.text)

## Take a look at the facets

In [ ]:
#parameters = osdd.findall('./{http://a9.com/-/spec/opensearch/1.1/}Url/{http://a9.com/-/spec/opensearch/extensions/parameters/1.0/}Parameter')

# Find geo+json url object
geojson_url_node = osdd.findall('.//{http://a9.com/-/spec/opensearch/extensions/parameters/1.0/}Parameter/..[@type="application/geo+json"]')

for param in geojson_url_node[0]:
    print(param.tag, param.attrib)



In [ ]:
# Take a look at the possible values for ecv

ecv_options = geojson_url_node[0].findall('.//{http://a9.com/-/spec/opensearch/extensions/parameters/1.0/}Option/..[@name="ecv"]')

for option in ecv_options[0]:
    print(option.tag, option.attrib)

## Make a search using a facet value

In [ ]:


# Retrieve cci collections
r = get_opensearch_request(f'{OPENSEARCH_URL}/request?parentIdentifier=cci&ecv=CLOUD')

print(r.status_code)

# Display the first collection in the response
first_coll = r.json()['features'][0]
pprint(first_coll)



In [ ]:
# Show links
pprint(first_coll['properties']['links'])



In [ ]:
# Show aggregations
pprint(first_coll['properties']['aggregations'])

# Using Aggregations